# Proyecto: Clasificación de tópicos de interés

## Exploración de datos


CC5113 - Aprendizaje Automático Bayesiano

Profesor: Pablo Guerrero

Autor: Martín Cornejo

## Paquetes necesarios

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pdb
import itertools
import operator

import pandas as pd

from nltk.stem.snowball import SpanishStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [2]:
datos=pd.read_csv('data_format.csv')
print(datos.shape)
datos.head()

(138, 2)


,Texto,Interes
0,"Ojalá obliguen a Piñera a cerrar Punta Peuco, ...",False
1,Piñera para crear base de apoyo moderada a su ...,True
2,@CNNChile MEMORIA 2014 Adimark: Piñera termina...,True
3,PPK y Piñera en privado habrían conversado alg...,False
4,Bachelet entregará el gobierno de Chile a Piñera,True


## Limpiando strings

### Reemplazar tildes, caracteres especiales, todo a minúsculas

In [3]:
strings = datos.ix[:,0]

def formatear(strings):
    tildes = ['á','é','í','ó','ú']
    vocales = ['a','e','i','o','u']

    # tildes
    for idx, vocal in enumerate(vocales):
        strings = strings.str.replace(tildes[idx],vocal)

    # caracteres especiales menos la ñ
    strings = strings.str.replace('[^a-zñA-Z ]', "")

    # todo a minusculas
    strings = pd.Series(list(map(lambda x: x.lower(), strings)))
    
    return strings

def oracionToStrArr(strings):
    strings_arr = list(map(lambda x: x.split(), strings))
    #pdb.set_trace()
    strings_arr = list(itertools.chain.from_iterable(strings_arr))    
    return strings_arr

print(formatear(strings).head())
formated_array_data = oracionToStrArr(formatear(strings))

0    ojala obliguen a piñera a cerrar punta peuco e...
1    piñera para crear base de apoyo moderada a su ...
2    cnnchile memoria  adimark piñera termina su go...
3    ppk y piñera en privado habrian conversado alg...
4     bachelet entregara el gobierno de chile a piñera
dtype: object


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


## Funciones auxiliares (conteo y ordenar repetidas)

In [4]:
def most_common(str_arr):
  # get an iterable of (item, iterable) pairs
  SL = sorted((x, i) for i, x in enumerate(str_arr))
  list_pairs = []
  #print('SL:', SL)
  groups = itertools.groupby(SL, key=operator.itemgetter(0))
    
  # auxiliary function to get "quality" for an item
  def _auxfun(g):
    item, iterable = g
    count = 0
    min_index = len(str_arr)
    for _, where in iterable:
      count += 1
      min_index = min(min_index, where)
    list_pairs.append((item, count))
    #print('item %r, count %r, minind %r' % (item, count, min_index))
    return count, -min_index

  return max(groups, key=_auxfun)[0], list_pairs

def aplicar_ordenar_str_arr(func, str_arr):
    arr = list(map(func, str_arr))
    common, pairs = most_common(arr)
    pares_filtrados = list(filter(lambda x: len(x[0]) > 3, pairs))
    common_sorted = sorted(pares_filtrados, key=lambda tup: tup[1], reverse=True)
    return common_sorted

## Separando data por clases

In [5]:
datos_interes = datos[datos.Interes == True]
#print(datos_interes.head())
str_interes = datos_interes.ix[:,0]

datos_no_interes = datos[datos.Interes == False]
#print(datos_no_interes.head())
str_no_interes = datos_no_interes.ix[:,0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


## Stemming por clases

In [6]:
stemmer_es = lambda x: SpanishStemmer().stem(x)

stem_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(stemmer_es,oracionToStrArr(formatear(str_interes))))
stem_no_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(stemmer_es,oracionToStrArr(formatear(str_no_interes))))

stem_interes_ordenado

0           (piñer, 75)
1       (president, 25)
2         (sebasti, 23)
3        (bachelet, 20)
4            (chil, 20)
5         (gobiern, 15)
6             (mand, 9)
7          (reunion, 8)
8            (activ, 6)
9             (asum, 6)
10           (cambi, 6)
11          (doming, 6)
12            (esta, 6)
13            (este, 6)
14           (macri, 6)
15           (mañan, 6)
16            (nuev, 6)
17            (pais, 6)
18        (asuncion, 5)
19           (cierr, 5)
20          (derech, 5)
21            (lleg, 5)
22            (ahor, 4)
23            (cerr, 4)
24          (chilen, 4)
25          (gobern, 4)
26           (mejor, 4)
27         (oficial, 4)
28            (peuc, 4)
29           (polit, 4)
             ...       
373           (serl, 1)
374         (sistem, 1)
375         (solemn, 1)
376       (solucion, 1)
377           (subs, 1)
378        (sucesor, 1)
379        (supuest, 1)
380        (suramer, 1)
381          (tambi, 1)
382           (tant, 1)
383    (temprani

In [7]:
stem_no_interes_ordenado

0               (piñer, 58)
1              (sebasti, 7)
2                 (bien, 6)
3                (cuand, 6)
4                 (esta, 6)
5                 (peuc, 6)
6                 (cerr, 5)
7                 (punt, 5)
8                 (ahor, 4)
9                 (chil, 4)
10               (cierr, 4)
11             (academi, 3)
12                (algo, 3)
13            (bachelet, 3)
14            (diplomat, 3)
15                (lleg, 3)
16               (penal, 3)
17                (pier, 3)
18               (quier, 3)
19                (reun, 3)
20                (abog, 2)
21               (afirm, 2)
22                (ayud, 2)
23                (buen, 2)
24     (cierrepuntapeuc, 2)
25             (convers, 2)
26               (culia, 2)
27              (ejecut, 2)
28            (eleccion, 2)
29               (empez, 2)
               ...         
295               (suci, 1)
296           (suddenly, 1)
297               (suel, 1)
298              (suert, 1)
299             (tam

## Lematización

In [8]:
def create_lemma_dict(filename):
   with open(filename, 'r') as document:
       lemma_dict = {}
       for line in document:
           if line.strip():  # avoid empty lines
               value, key = line.split(None, 1) # 'None' means 'all whitespace', which is the default
               key = key.rstrip() # rstrip() to get rid of \r and \n
               lemma_dict[key] = value # adding the flections as keys to the dict
               lemma_dict[value] = value # adding also the base word as a key
   return lemma_dict

def query_word(lemma_dict):
   word = input("\nDame una palabra en español -> ")
   try:
      lemma = lemma_dict[word]
      print("__your happy lemma is__: {}".format(lemma))
   except KeyError:
      print("This word is not in the dictionary!")
   return query_word(lemma_dict)

def lemmatiser(dict):
    def lookup(word):
        try:
            lemma = dict[word]
        except:
            lemma = word
        
        return lemma
    
    return lookup

## Lematizando por clases

In [9]:
resource_file = 'lemmatization-es.txt'
lemmatiser_es = lemmatiser(create_lemma_dict(resource_file))

#pdb.set_trace()

datos_interes = datos[datos.Interes == True]
#print(datos_interes.head())
str_interes = datos_interes.ix[:,0]

datos_no_interes = datos[datos.Interes == False]
#print(datos_no_interes.head())
str_no_interes = datos_no_interes.ix[:,0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


In [10]:
lema_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(lemmatiser_es, oracionToStrArr(formatear(str_interes))))
lema_no_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(lemmatiser_es, oracionToStrArr(formatear(str_no_interes))))

#lema_interes_ordenado

In [11]:
#lema_no_interes_ordenado

## Extracción de características usando sklearn

In [12]:
stop_words_es = np.genfromtxt('stop_words_es.txt', dtype='str')
stop_words_es = formatear(pd.Series(stop_words_es))
stop_words_es = list(map(lambda x: x, stop_words_es))

In [13]:
tf_vectorizer = CountVectorizer(min_df=2, max_features=15, stop_words=stop_words_es)
data = formatear(strings)
#pdb.set_trace()
dtm_tf = tf_vectorizer.fit_transform(data)

## Aplicando Latent Dirichlet Allocation

In [14]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        #pdb.set_trace()
        message += " ".join([feature_names[i] + " " + str(int(topic[i])) + " "
                             for i in topic.argsort()[:-n_top_words - 1:-1]])        
        print(message)
    print()

In [15]:
lda = LatentDirichletAllocation(n_components=3, max_iter=100,
                                learning_method='online',
                                learning_offset=10,
                                batch_size=10,
                                random_state=1)

In [16]:
n_top_words = 6
lda.fit(dtm_tf)
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: piñera 30  sebastian 30  presidente 21  chile 10  mando 9  reunion 8 
Topic #1: bachelet 23  piñera 17  peuco 10  punta 9  gobierno 7  cierre 7 
Topic #2: piñera 85  chile 13  gobierno 8  mañana 8  domingo 0  reunion 0 



In [17]:
pyLDAvis.sklearn.prepare(lda, dtm_tf, tf_vectorizer)

/home/martin/.local/lib/python3.5/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      40.487083        1       1 -0.212636  0.077042
2      32.185653        1       2 -0.008267 -0.145747
1      27.327264        1       3  0.220902  0.068705, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
0     Default   26.000000    bachelet   26.000000  15.0000  15.0000
14    Default   29.000000   sebastian   29.000000  14.0000  14.0000
10    Default  122.000000      piñera  122.000000  13.0000  13.0000
11    Default   22.000000  presidente   22.000000  12.0000  12.0000
9     Default   12.000000       peuco   12.000000  11.0000  11.0000
12    Default   11.000000       punta   11.000000  10.0000  10.0000
3     Default    8.000000      cierre    8.000000   9.0000   9.0000
5     Default   16.000000    gobierno   16.000000   8.0000   8.0000
2     Default   22.000000       chile   22.000000   7.0000   7.0000
8     Default    7.000000      mañana    7.000000   6.0000   6.0000
7     Default    9.000000       mando    9.000000   5.0000   5.0000
13    Default    8.000000     reunion    8.000000   4.0000   4.0000
6     Default    7.000000       macri    7.000000   3.0000   3.0000
1     Default    6.000000      cambio    6.000000   2.0000   2.0000
4     Default    6.000000     domingo    6.000000   1.0000   1.0000
14     Topic1   29.278149   sebastian   29.950759   0.8815  -1.4810
11     Topic1   21.368391  presidente   22.034514   0.8735  -1.7960
7      Topic1    8.948139       mando    9.612688   0.8325  -2.6664
13     Topic1    7.991075     reunion    8.650220   0.8249  -2.7795
6      Topic1    6.919996       macri    7.574833   0.8138  -2.9234
1      Topic1    6.074162      cambio    6.741718   0.7999  -3.0538
4      Topic1    5.973653     domingo    6.656586   0.7959  -3.0705
2      Topic1   10.633129       chile   22.505432   0.1544  -2.4939
10     Topic1   29.562438      piñera  122.346487  -0.5162  -1.4714
8      Topic1    0.343200      mañana    7.659541  -2.2012  -5.9273
3      Topic1    0.324578      cierre    8.524968  -2.3640  -5.9831
12     Topic1    0.326519       punta   11.004132  -2.6134  -5.9771
9      Topic1    0.326264       peuco   12.083553  -2.7077  -5.9779
5      Topic1    0.336743    gobierno   16.353341  -2.9787  -5.9463
0      Topic1    0.342488    bachelet   26.301227  -3.4369  -5.9294
8      Topic2    6.907981      mañana    7.659541   1.0304  -2.6957
10     Topic2   73.333978      piñera  122.346487   0.6218  -0.3334
2      Topic2   11.488981       chile   22.505432   0.4613  -2.1870
5      Topic2    7.409893    gobierno   16.353341   0.3420  -2.6256
4      Topic2    0.300277     domingo    6.656586  -1.9650  -5.8315
1      Topic2    0.287158      cambio    6.741718  -2.0224  -5.8761
6      Topic2    0.291380       macri    7.574833  -2.1243  -5.8615
13     Topic2    0.296023     reunion    8.650220  -2.2413  -5.8457
3      Topic2    0.288675      cierre    8.524968  -2.2518  -5.8709
7      Topic2    0.290404       mando    9.612688  -2.3659  -5.8649
12     Topic2    0.288172       punta   11.004132  -2.5088  -5.8726
9      Topic2    0.287429       peuco   12.083553  -2.6050  -5.8752
11     Topic2    0.292278  presidente   22.034514  -3.1890  -5.8585
0      Topic2    0.293410    bachelet   26.301227  -3.3622  -5.8546
14     Topic2    0.294338   sebastian   29.950759  -3.4889  -5.8514
0      Topic3   25.665329    bachelet   26.301227   1.2728  -1.2196
9      Topic3   11.469861       peuco   12.083553   1.2452  -2.0250
12     Topic3   10.389441       punta   11.004132   1.2398  -2.1240
3      Topic3    7.911715      cierre    8.524968   1.2226  -2.3964
5      Topic3    8.606705    gobierno   16.353341   0.6554  -2.3122
10     Topic3   19.450071      piñera  122.346487  -0.5417  -1.4969
4      Topic3    0.382657     domingo    6.656586  -1.5589  -5.4254
1      Topic3  